In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

device_names = {
    1: "Strymon Cloudburst",
    2: "Strymon Timeline",
    3: "Chase Bliss Mood MK II",
    4: "Chase Bliss Blooper",
    5: "Chase Bliss Lost And Found",
    6: "Source Audio EQ2",
    7: "GT 1000 Core"
}

def generate_midi_message():
    channel = np.random.randint(1, 8)
    return {
        'Timestamp': datetime.utcnow(),
        'Channel': channel,
        'Device': device_names[channel],
        'CC Number': np.random.randint(0, 128),
        'CC Value': np.random.randint(0, 128),
        'Program Number': np.random.randint(0, 128)
    }

# Generate 100 messages
midi_data = [generate_midi_message() for _ in range(1000)]

df = pd.DataFrame(midi_data)



In [3]:
df.head()

,Timestamp,Channel,Device,CC Number,CC Value,Program Number
0,2025-12-30 00:37:33.466724,6,Source Audio EQ2,118,117,43
1,2025-12-30 00:37:33.466738,4,Chase Bliss Blooper,15,33,41
2,2025-12-30 00:37:33.466744,7,GT 1000 Core,72,84,97
3,2025-12-30 00:37:33.466749,2,Strymon Timeline,38,91,93
4,2025-12-30 00:37:33.466754,2,Strymon Timeline,27,117,51


In [4]:
!pip

zsh:1: command not found: pip


In [23]:
from pythonosc import dispatcher, osc_server
import threading

# Global counter
counter = 0

def amp_handler(addr, *args):
    global counter
    counter += 1
    print(f"[Python] #{counter} Received {addr}: {args[0]}")

# Setup dispatcher
disp = dispatcher.Dispatcher()
disp.map("/hello", amp_handler)

# Create server
server = osc_server.ThreadingOSCUDPServer(("127.0.0.1", 9001), disp)
print("[Python] Listening on port 9001...")

# Run server in a separate thread
server_thread = threading.Thread(target=lambda: server.serve_forever(poll_interval=0.1))
server_thread.start()




[Python] Listening on port 9001...


In [24]:
print("[Python] Shutting down server...")
server.shutdown()
server.server_close()
server_thread.join()
print("[Python] Server stopped.")

[Python] Shutting down server...
[Python] Server stopped.


In [ ]:
from pythonosc import dispatcher, osc_server

def print_handler(addr, *args):
    print("Received:", addr, args)

disp = dispatcher.Dispatcher()
disp.map("/hello", print_handler)

server = osc_server.ThreadingOSCUDPServer(("127.0.0.1", 9001), disp)
print("Listening on 127.0.0.1:9001...")
server.serve_forever()


Listening on 127.0.0.1:9001...
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello ('Hello World',)
Received: /hello 

In [ ]:
print("[Python] Shutting down server...")
server.shutdown()
server.server_close()
server_thread.join()
print("[Python] Server stopped.")

In [ ]:
import sounddevice as sd


def audio_callback(indata, frames, time, status):
    rms = np.sqrt(np.mean(indata**2))
    fft = np.fft.rfft(indata[:,0])
    centroid = np.sum(np.abs(fft) * np.arange(len(fft))) / (np.sum(np.abs(fft))+1e-6)
    # save to database or send via OSC
    print("RMS:", rms, "Centroid:", centroid)

stream = sd.InputStream(channels=1, callback=audio_callback, blocksize=1024, samplerate=44100)
stream.start()


In [ ]:
import sounddevice as sd
print(sd.query_devices())

In [ ]:
import sqlite3

DB_PATH = "music_data.db"

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Count rows
cursor.execute("SELECT COUNT(*) FROM audio_features")
count = cursor.fetchone()[0]
print(f"Total rows in audio_features: {count}")

# Preview last 5 rows
cursor.execute("""
    SELECT timestamp, rms_db, centroid FROM audio_features
    ORDER BY timestamp DESC
    LIMIT 5
""")
rows = cursor.fetchall()

if rows:
    print("Last 5 entries:")
    for r in rows:
        print(r)
else:
    print("No data yet!")

conn.close()
